In [1]:
import pandas as pd

In [5]:
import numpy as np

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
import ast

In [13]:
movies = pd.read_csv(r'C:\Users\Administrator\Desktop\Data_sets\tmdb_5000_movies.csv')

In [15]:
credits = pd.read_csv(r'C:\Users\Administrator\Desktop\Data_sets\tmdb_5000_credits.csv')

In [17]:
movies = movies.merge(credits, on='title')

In [19]:
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [21]:
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [23]:
def get_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L

In [25]:
def collapse(L):
    return " ".join(L)

movies.dropna(inplace=True)

movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)
movies['cast'] = movies['cast'].apply(lambda x: convert(x)[:3])
movies['crew'] = movies['crew'].apply(get_director)

In [27]:
movies['tags'] = movies['overview'] + " " + \
                 movies['genres'].apply(collapse) + " " + \
                 movies['keywords'].apply(collapse) + " " + \
                 movies['cast'].apply(collapse) + " " + \
                 movies['crew'].apply(collapse)

new_df = movies[['movie_id', 'title', 'tags']]

In [31]:
new_df = new_df.copy()
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())

cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(new_df['tags']).toarray()

In [33]:
similarity = cosine_similarity(vectors)

In [35]:
def recommend(movie):
    movie = movie.lower()
    if movie not in new_df['title'].str.lower().values:
        return ["Movie not found in database."]
    idx = new_df[new_df['title'].str.lower() == movie].index[0]
    distances = list(enumerate(similarity[idx]))
    movies_list = sorted(distances, reverse=True, key=lambda x: x[1])[1:6]

    recommended_movies = []
    for i in movies_list:
        recommended_movies.append(new_df.iloc[i[0]].title)
    return recommended_movies

# Example Test
print(recommend('Avatar'))

['Aliens', 'Moonraker', 'Alien', 'Alien³', 'Silent Running']
